In [ ]:
import os

from decimal import Decimal
import numpy as np
from numpy import genfromtxt
from numpy import savetxt
import glob
import rea
import importlib
import gc
import matplotlib.pyplot as plt
import math

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import scale
import pandas as pd

import keras
from keras.models import load_model
from keras.models import Sequential, load_model
from keras.layers.core import Dense, Flatten
from keras.layers import Input, Concatenate
from keras.callbacks import ModelCheckpoint

from platform import python_version

import tensorflow as tf

rmse = tf.keras.metrics.RootMeanSquaredError()


In [ ]:
print('Loading original model and activation function:')
leakyrelu=lambda x: tf.keras.activations.relu(x, alpha=0.2)
Original=load_model("./originalModel.h5",custom_objects={'<lambda>': leakyrelu})

In [ ]:
print('Loading orignal dataset:')
ds=[]
orden=[]
total_samples = 0
for filename in glob.glob("./dataSet/NNTrainingTesting-3p-3c-v7_*.csv"):
    temp=filename[43:-1].split("_")
    DP=temp[:1]
    DP=''.join(DP)
    DP=int(DP)
    orden.append(DP)
    if DP >= 100:
        params = np.array(filename[49+2:-4].split("_"), dtype=np.float32)
    elif DP >= 10:
        params = np.array(filename[49+1:-4].split("_"), dtype=np.float32)
    else:
        params = np.array(filename[49:-4].split("_"), dtype=np.float32)  
    data = genfromtxt(filename, delimiter='\n')
    row = np.concatenate((data, params), axis=0)
    ds.append(row)
    total_samples+=1
    
print("Total samples: ", total_samples)

ds = np.array(ds, dtype=np.float32)
orden=np.array(orden, dtype=int)

print(ds.shape)

In [ ]:
#Prepare dataset

features=np.array(ds[:,:600])

scaler=StandardScaler()
scaler.fit(features)

trainData = features[:400,:]
valData = features[400:,:]

trainData=scaler.transform(trainData)
valData=scaler.transform(valData)

valData11=np.array(valData[:,0:598:3])
valData12=np.array(valData[:,1:599:3])
valData13=np.array(valData[:,2:600:3])

valDataTensor=tf.convert_to_tensor(valData)
valData11Tensor=tf.convert_to_tensor(valData11)
valData12Tensor=tf.convert_to_tensor(valData12)
valData13Tensor=tf.convert_to_tensor(valData13)

In [ ]:
#Call Gradient Tape to perform Automatic Differentiation

with tf.GradientTape() as grad:
    grad.watch([valData11Tensor,valData12Tensor,valData13Tensor])
    out=Original([valData11Tensor,valData12Tensor,valData13Tensor])

gradients=grad.gradient(out,[valData11Tensor,valData12Tensor,valData13Tensor])

In [ ]:
#Separate from independent layers
part1=np.array(gradients[0],dtype=np.float32)
part2=np.array(gradients[1],dtype=np.float32)
part3=np.array(gradients[2],dtype=np.float32)
newGradients=np.concatenate((part1,part2,part3),axis=-1)

In [ ]:
#Compute metric with gradients

absgrad=np.abs(newGradients)

importancia=np.zeros((600,1))

for i in range(600):
    importancia[i,0]=np.mean(absgrad[:,i])

In [ ]:
plt.hist(importancia)
plt.show()

In [ ]:
#Select inputs with mean higher than a given threslhold
vips=[]
for i in range(600):
    if importancia[i,0]>0.0165:
        vips.append(i)
vips=np.array(vips,dtype=int)
vips=np.sort(vips)
print(len(vips))
print(vips)

In [ ]:
#Locate the input to represent it

componente=[]
fila=[]
punto=[]

for i in range(len(vips)):
    if vips[i]<200:
        real=vips[i]*3
    elif (vips[i]>=200)&(vips[i]<400):
        real=(vips[i]-200)*3+1
    else:
        real=(vips[i]-400)*3+2

    resto=np.remainder(real,3)
    componente.append(resto+1)
    for j in range(10):
        if (real<(600-60*j))&(real>=(600-60*(j+1))):
            fila.append(j)
    for z in range(20):
        if (real>=(60*(9-fila[i])+3*z))&(real<(60*(9-fila[i])+3*(z+1))):
            punto.append(z)
  

In [ ]:
#Print inputs' locations

deltax=0.105
deltaz=0.5555558

posicion=np.zeros((len(vips),2))
for i in range(len(vips)):
    posicion[i,0]=-punto[i]*deltax
    posicion[i,1]=-2.5+fila[i]*deltaz
    
    
line1=np.array([[-5,7],[0,0]])
line2=np.array([[0,0],[0.5,-4]])
plt.scatter(posicion[:,1],posicion[:,0],color='r')
plt.plot(line1[0],line1[1],'k--')
plt.plot(line2[0],line2[1],'k--')
plt.ylim(0.5,-4)
plt.xlim(7,-5)
plt.xlabel('z [m]',fontsize=15)
plt.ylabel('r [m]',fontsize=15)
plt.show()

In [ ]:
#Complete the sampling point with the remaining velocity components of each selected input

datos=[]
completos=np.ones(len(vips)*3)*-1
for i in range(len(vips)):
    if vips[i]<200:
        real=vips[i]*3
    elif (vips[i]>=200)&(vips[i]<400):
        real=(vips[i]-200)*3+1
    else:
        real=(vips[i]-400)*3+2
        
    resto=np.remainder(real,3)
    if (real==completos).any():
        print('Sampling point repeated')
    else:
        if resto==0:
            col=[ds[:,real],ds[:,real+1],ds[:,real+2]]
            col=np.array(col,dtype=float)
            col2=col.transpose()
            completos[3*i]=real
            completos[3*i+1]=real+1
            completos[3*i+2]=real+2
        elif resto==1:
            col=[ds[:,real-1],ds[:,real],ds[:,real+1]]
            col=np.array(col,dtype=float)
            col2=col.transpose()
            completos[3*i]=real-1
            completos[3*i+1]=real
            completos[3*i+2]=real+1
        elif resto==2:
            col=[ds[:,real-2],ds[:,real-1],ds[:,real]]
            col=np.array(col,dtype=float)
            col2=col.transpose()
            completos[3*i]=real-2
            completos[3*i+1]=real-1
            completos[3*i+2]=real
        if i==0:
            datos=col2
        else:
            datos=np.append(datos,col2,axis=1)

            datos=np.array(datos,dtype=float)
            
print(datos.shape)
inputs=datos.shape[1]
print(inputs/3)

In [ ]:
#Reduce dataset leaving only the selected sampling points

labels=np.array(ds[:,600:])/5000

scaler=StandardScaler()
scaler.fit(datos)

train_final_input = datos[:400,:]
test_final_input = datos[400:,:]
train_final_label=labels[:400,:]
test_final_label=labels[400:,:]

train_final_input=scaler.transform(train_final_input)
test_final_input=scaler.transform(test_final_input)

trainfi_Vx=np.array(train_final_input[:,0:inputs-2:3])
trainfi_Vy=np.array(train_final_input[:,1:inputs-1:3])
trainfi_Vz=np.array(train_final_input[:,2:inputs:3])

testfi_Vx=np.array(test_final_input[:,0:inputs-2:3])
testfi_Vy=np.array(test_final_input[:,1:inputs-1:3])
testfi_Vz=np.array(test_final_input[:,2:inputs:3])

trainfi_Vx_pd=pd.DataFrame(trainfi_Vx)
trainfi_Vy_pd=pd.DataFrame(trainfi_Vy)
trainfi_Vz_pd=pd.DataFrame(trainfi_Vz)
testfi_Vx_pd=pd.DataFrame(testfi_Vx)
testfi_Vy_pd=pd.DataFrame(testfi_Vy)
testfi_Vz_pd=pd.DataFrame(testfi_Vz)


trainfl_pd=pd.DataFrame(train_final_label)
testfl_pd=pd.DataFrame(test_final_label)

In [ ]:
#Create MLP with initial layer adapted to new number of inputs

I1=Input(shape=(trainfi_Vx.shape[1],))
I2=Input(shape=(trainfi_Vy.shape[1],))
I3=Input(shape=(trainfi_Vz.shape[1],))

h1_p1=Dense(trainfi_Vx.shape[1],activation=leakyrelu)(I1)
h1_p2=Dense(trainfi_Vy.shape[1],activation=leakyrelu)(I2)
h1_p3=Dense(trainfi_Vz.shape[1],activation=leakyrelu)(I3)
h1=Concatenate()([h1_p1,h1_p2,h1_p3])
h2=Dense(500,activation=leakyrelu)(h1)
hdp1=keras.layers.Dropout(0.5)(h2)
h3=Dense(420,activation=leakyrelu)(hdp1)
hdp2=keras.layers.Dropout(0.35)(h3)
h4=Dense(360,activation=leakyrelu)(hdp2)
hdp3=keras.layers.Dropout(0.25)(h4)
h5=Dense(300,activation=leakyrelu)(hdp3)
hdp4=keras.layers.Dropout(0.15)(h5)
h6=Dense(150,activation=leakyrelu)(hdp4)
h7=Dense(75,activation=leakyrelu)(h6)
h8=Dense(32,activation=leakyrelu)(h7)
Out=Dense(3)(h8)

model=keras.models.Model(inputs=[I1,I2,I3],outputs=Out)


model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), metrics=[rmse, 'mae'])
model.summary()

In [ ]:
#Train NN

history = model.fit(
    [trainfi_Vx_pd,trainfi_Vy_pd,trainfi_Vz_pd], trainfl_pd,
    epochs=1500,
    verbose=1,
    validation_split = 0.35)

In [ ]:
def plot_loss(history,maximo):
  plt.figure(figsize=(16,8))
  plt.plot(history.history['loss'], label='loss')
  plt.plot(history.history['val_loss'], label='val_loss')
  plt.ylim([0, maximo])
  plt.xlabel('Epoch')
  plt.ylabel('Error')
  plt.legend()
  plt.grid(True)

In [ ]:
#Print training
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
plot_loss(history,0.05)

In [ ]:
#Evaluate model
results= model.evaluate([testfi_Vx_pd,testfi_Vy_pd,testfi_Vz_pd], testfl_pd, verbose=1)

In [ ]:
#Generate predictions
predicciones=model.predict([testfi_Vx_pd,testfi_Vy_pd,testfi_Vz_pd])
for i in range(3):
    plt.scatter(predicciones[:,i]*5000,labels[400:,i]*5000)
    plt.title('component S %i' %i)
    plt.show()

In [ ]:
#Save model and predictions
model.save("./NN_M6b.h5")
scaledPred=predicciones*5000
np.savetxt('./Preds_M6b.csv', scaledPred, delimiter=';', fmt='%0.6f')